In [ ]:
# ---- Mongo smoke test (using your animal_shelter.py)
import pandas as pd
from animal_shelter import AnimalShelter

db = AnimalShelter()          # your class embeds aacuser/aacuser01 and aac.animals

rows = db.read({})            # your read() returns a Python list already
print("Total docs:", len(rows))

df = pd.DataFrame(rows)
if "_id" in df.columns:
    df = df.drop(columns=["_id"])   # drop ObjectId for display
df.head(10)


In [ ]:
%pip install dash==2.17.1 jupyter-dash dash-leaflet pandas plotly pymongo dnspython


In [ ]:
from jupyter_dash import JupyterDash

# clear any stuck Dash server threads so ports can be reused
try:
    JupyterDash._server_threads.clear()
except Exception:
    JupyterDash._server_threads = {}


In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
import re

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "aacuser01"

# Connect to database via CRUD Module
db = AnimalShelter()


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({})).head(500)


# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__, suppress_callback_exceptions=True)

app.layout = html.Div([
    html.Img(src=app.get_asset_url('grazioso_logo.png'), style={"height": "60px"}),
    html.Div("Dashboard by Brandon Armitage • 2025-08-20", style={"fontStyle": "italic"}),

    # session flag for auth
    dcc.Store(id="auth-ok", storage_type="session"),

    # login box
    html.Div(
        [
            html.H4("Dashboard Login"),
            dcc.Input(id="login-username", type="text", placeholder="Username"),
            dcc.Input(id="login-password", type="password", placeholder="Password", style={"marginLeft": "8px"}),
            html.Button("Login", id="login-btn", n_clicks=0, style={"marginLeft": "8px"}),
            html.Span(id="login-msg", style={"marginLeft": "12px"})
        ],
        id="login-box",
        style={"marginTop": "10px", "marginBottom": "10px"}
    ),

    # this is where the dashboard will be injected AFTER login
    html.Div(id="dashboard-content")
])
def build_dashboard():
    return [
        html.Center(html.B(html.H1('CS-340 Dashboard'))),

        dcc.Dropdown(
            id="filter-type",
            options=[
                {"label": "Reset (All Animals)", "value": "reset"},
                {"label": "Water Rescue", "value": "water"},
                {"label": "Mountain/Wilderness Rescue", "value": "mountain"},
                {"label": "Disaster / Individual Tracking", "value": "disaster"},
            ],
            value="reset",
            clearable=False,
            style={"width": "400px"}
        ),

        html.Div(
            style={"display": "flex", "gap": "24px", "alignItems": "center", "marginTop": "12px"},
            children=[
                dcc.RadioItems(
                    id="type-filter",
                    options=[{"label": "All", "value": "all"},
                             {"label": "Dog", "value": "Dog"},
                             {"label": "Cat", "value": "Cat"}],
                    value="all",
                    inline=True
                ),
                dcc.Input(id="breed-search", type="text", placeholder="Search breed…", style={"width": "250px"}),
            ],
        ),

        html.Hr(),
        html.H3("Unfiltered Animal Data Table"),

        dash_table.DataTable(
            id='datatable-id',
            columns=[{"name": c, "id": c} for c in df.columns],
            data=df.to_dict('records'),
            page_size=10,
            filter_action="native",
            sort_action="native",
            row_selectable="single",
            selected_rows=[0],
            style_table={"overflowX": "auto"},
            style_cell={"textAlign": "left", "minWidth": "100px", "width": "150px", "maxWidth": "220px"}
        ),

        html.Br(),
        html.Div(id='graph-id'),
        html.Br(),
        html.Div(id='map-id'),
    ]

  


RESCUE_RX = {
    "reset": None,
    "water": r"(Retriever|Newfoundland)",
    "mountain": r"(Collie|Shepherd)",
    "disaster": r"(Hound|Terrier)",
}

@app.callback(
    Output("datatable-id", "data"),
    Input("filter-type", "value"),      # rescue type
    Input("type-filter", "value"),      # Dog/Cat/All
    Input("breed-search", "value"),     # free text
)
def update_table(rescue, species, breed_text):
    # Build a Mongo query for your CRUD module
    query = {}

    # Dog/Cat/All
    if species and species != "all":
        query["animal_type"] = species

    # Rescue-type regex
    parts = []
    if rescue and RESCUE_RX.get(rescue):
        parts.append(RESCUE_RX[rescue])

    # Optional breed search text (case-insensitive)
    if breed_text:
        parts.append(re.escape(breed_text.strip()))

    if parts:
        # join with '.*' so both conditions can appear anywhere in the breed string
        rx = "(?i)" + ".*".join(parts)        # (?i) -> case-insensitive
        query["breed"] = {"$regex": rx}

    # Query Mongo via your AnimalShelter class
    rows = db.read(query)                     # returns list[dict]
    dff = pd.DataFrame(rows).head(500)
    if "_id" in dff.columns:
        dff.drop(columns=["_id"], inplace=True)

    return dff.to_dict("records")
    
#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    Output('graph-id', "children"),
    Input('datatable-id', "derived_virtual_data")
)
def update_graphs(viewData):
    dff = pd.DataFrame(viewData) if viewData else df.copy()

    # choose a categorical column that exists
    for cand in ['breed', 'animal_type', 'outcome_type']:
        if cand in dff.columns:
            target = cand
            break
    else:
        return html.Div("No suitable categorical column found for chart.")

    # count and set column names deterministically
    vc = dff[target].astype(str).replace(['', 'nan', 'None'], np.nan).dropna().value_counts()
    top = vc.reset_index().head(10)
    top.columns = [target, 'count']   # <-- fix: make sure we have [name, count]

    if top.empty:
        return html.Div("No data to display in chart for the current view.")

    fig = px.pie(top, values='count', names=target, title=f"Top {target.replace('_',' ').title()} (current view)")
    return dcc.Graph(figure=fig)


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    Input('datatable-id', 'selected_columns')
)
def update_styles(selected_columns):
    if not selected_columns:   # <--- prevent NoneType error
        return []
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

    

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(rows, selected):
    # If there is no data yet, show a blank basemap
    if not rows:
        return dl.Map(
            style={'width': '100%', 'height': '500px'},
            center=[30.75, -97.48], zoom=10,
            children=[dl.TileLayer(id="base-layer-id")]
        )

    dff = pd.DataFrame(rows)
    row = 0
    if selected and len(selected) > 0:
        try:
            row = int(selected[0])
        except Exception:
            row = 0

    # Clamp row index to valid range
    row = max(0, min(row, len(dff) - 1))

    # Your CSV/Mongo field order uses lat/lon at columns 13, 14
# Prefer named columns; fall back to positional if needed
    if {"location_lat", "location_long"}.issubset(dff.columns):
        lat = dff.loc[row, "location_lat"]
        lon = dff.loc[row, "location_long"]
    else:
        lat = dff.iloc[row, 13]
        lon = dff.iloc[row, 14]

    # If missing coordinates, just show the basemap centered on Austin
    if pd.isna(lat) or pd.isna(lon):
        return dl.Map(
            style={'width': '100%', 'height': '500px'},
            center=[30.75, -97.48], zoom=10,
            children=[dl.TileLayer(id="base-layer-id")]
        )

    # Build the map with a single marker for the selected row
    return dl.Map(
        style={'width': '100%', 'height': '500px'},
        center=[lat, lon], zoom=10,
        children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(
                position=[lat, lon],
                children=[
                    dl.Tooltip(str(dff.iloc[row, 4])),  # breed
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(str(dff.iloc[row, 9]))      # name
                    ])
                ]
            )
        ]
    )

@app.callback(
    Output("auth-ok", "data"),
    Output("login-msg", "children"),
    Input("login-btn", "n_clicks"),
    State("login-username", "value"),
    State("login-password", "value"),
    prevent_initial_call=True
)
def do_login(n, u, p):
    if u == username and p == password:
        return True, html.Span("Login successful!", style={"color": "green", "fontWeight": "bold"})
    return None, html.Span("Invalid credentials.", style={"color": "crimson", "fontWeight": "bold"})

@app.callback(
    Output("dashboard-content", "children"),
    Output("login-box", "style"),
    Input("auth-ok", "data")
)
def render_content(authed):
    if authed:
        # hide login, show dashboard
        return build_dashboard(), {"display": "none"}
    # show login, no dashboard yet
    return [], {"marginTop": "10px", "marginBottom": "10px"}



app.run_server(mode="inline", debug=False, port=8062) 




C:\Users\holen\anaconda3\Lib\site-packages\dash\dash.py:556: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.

